In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install docx to read incoming new docx file
!pip install python-docx

     |████████████████████████████████| 5.6 MB 4.7 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184508 sha256=a2c4b86f794e7b48902c8d30a93e2680e3c0a8a0a159f90237e1f83be4b08a9e
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx


In [3]:
import os

In [4]:
os.chdir('/content/drive/MyDrive/capstone/') #Ujjval

In [5]:
# import
import warnings
import os
import pandas as pd 
import numpy as np

from google.colab import files
import time

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import pyplot
import seaborn as sns
import tensorflow 
print(tensorflow.__version__)
# suppress display of warnings
warnings.filterwarnings('ignore')
# display all dataframe columns
pd.options.display.max_columns = None

# to set the limit to 3 decimals
pd.options.display.float_format = '{:.7f}'.format

# display all dataframe rows
pd.options.display.max_rows = None

2.5.0


In [62]:
#path = "/content/drive/MyDrive/AI-ML/Data/TestAtricle1MarketingMedium.docx"
#path = "/content/drive/MyDrive/AI-ML/Data/TestArticle3_Finance_Large.docx"
#path = "/content/drive/MyDrive/capstone/Data_Test_Articles/TestArticle3_Finance_Large.docx" #Ujjval
path = "/content/drive/MyDrive/capstone/Data_Test_Articles/TestArticle 4.docx" #Ujjval
#https://managemagazine.com/blogtalk/management/henry-mintzberg-half-truths-management/
'''
### read in the .docx file extention
'''
import docx

doc = docx.Document(path)
doc_data = [p.text for p in doc.paragraphs if p.text]

doc_data_str = ''
for p in doc.paragraphs:
  doc_data_str  += str(p.text)   

df = pd.DataFrame({'Article_Text': [doc_data_str]})

In [63]:
###########################################################
#### The below function is on Text Preprocessing File

#Clean up the new article using same function as done for train-test datasets
from bs4 import BeautifulSoup
import re

def str_strip(text):
    soup = BeautifulSoup(text, "html.parser") 
    text = re.sub('\[[^]]*\]', '', soup.get_text())
    text = re.sub('\nAuthor Abstract\n', '', text)
    text = re.sub('(?=Paper Information)(?s)(.*$)', '', text)
    text = text.strip()
    text = re.sub('\n', ' ', text)
    text = re.sub('\t', ' ', text)
    text = re.sub(u'\xa0', u' ', text)
    text = re.sub('—', ' ', text)  
    pattern=r"[^a-zA-z0-9\s,.;!()&$\-']"
    text=re.sub(pattern,'',text)
    text = re.sub('/\s\s+/g', ' ', text) #remove multiple spaces
    text = re.sub('/  +/g', ' ',text)
    read_pattern1 =r'\d min read'
    read_pattern2 =r'min read'
    translation_pattern = r'This article was translated from our Spanish edition using AI technologies. Errors may exist due to this process.'
    opinion_pattern = 'Opinions expressed by Entrepreneur contributors are their own.'
    date_pattern = r'^(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+(\d+)\,\s+(\d+)\s+' 
    text=re.sub(read_pattern1,'',text)
    text=re.sub(read_pattern2,'',text)
    text=re.sub(translation_pattern,'',text)
    text=re.sub(opinion_pattern,'',text)
    text=re.sub(date_pattern,'',text)
    return text

In [64]:
df['Cleaned_Article']=df['Article_Text'].apply(str_strip)

In [65]:
df['Cleaned_Article']=df['Article_Text'].apply(str_strip)
article_to_predict = df['Cleaned_Article'][0]

In [66]:
article_to_predict

'Henry Mintzberg about some Half-Truths of ManagementBy  - March 31, 2017SHARE   There are no whole truths; all truths are half-truths (Alfred North Whitehead). So here are a few in management.WE LIVE IN TIMES OF GREAT CHANGEHave you heard this before say in the last hour Did you know that when a laptop detects a CEO about to type a speech, it automatically enters We live in times of great change.  Why bother the CEO to type it again, since just about every management speech in the past few decades has begun with this line. That never changes.Do we really live in times of great change Look around and tell me whats changed fundamentally. Your food, your furniture, your friends, your fixations Are you wearing a tie, or high heels How come because you always have How about your car Under the hood is probably a four-cycle, internal combustion engine. That was in the Model T Ford.When you got dressed this morning, did you say to yourself If we live in times of great change, how come we are 

#Dimentions of articles - including Readability,Polarity,Subjectivity

In [67]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [68]:
!pip install textstat

In [69]:
! pip install textblob

In [70]:
import textstat 
import textblob
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [71]:
from textblob import TextBlob

In [72]:
stop = stopwords.words('english')

In [73]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

In [74]:
df['Word_Count']=df['Cleaned_Article'].apply(lambda x:len(x.split(' ')))
df['Char_Count'] = df['Cleaned_Article'].str.len()
df['Avg_Word_length'] = df['Cleaned_Article'].apply(lambda x: avg_word(x))
df['Stopwords_Count'] = df['Cleaned_Article'].apply(lambda x: len([x for x in x.split() if x in stop]))
df['numerics_Count'] = df['Cleaned_Article'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df['Upper_Count'] = df['Cleaned_Article'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
df['Syllable_Count'] = df['Cleaned_Article'].apply(lambda x: textstat.syllable_count(x))
df['Sentence_Count'] = df['Cleaned_Article'].apply(lambda x: textstat.sentence_count(x))
df['Difficult_Words_Count'] = df['Cleaned_Article'].apply(lambda x: textstat.difficult_words(x))
df['length'] = df['Cleaned_Article'].astype(str).apply(len)

In [75]:

df['Article_Size'] = df['Word_Count'].apply(lambda x: 'Small' if x <= 500 else ('Medium' if x <1000 else 'Large'))

In [76]:
df['Difficult_Words_%'] = (df['Difficult_Words_Count'] / df['Word_Count'])*100

In [77]:
#Difficult words (two-syllables) <10 - Basic 11-25 - Elementary 25-40 - Intermediate 40< - Advanced

In [78]:
#convertng into category
def Difficult(x):
    global ret    
    if (x < 10.0):
        ret = "Basic"  
    elif ((x > 10.0)and (x<= 25.0)):
        ret = "Elementary"
    elif ((x > 25.0)and (x<= 40.0)):
        ret = "Intermediate"
    elif (x > 40.0):
        ret = "Advanced"
    return ret

In [79]:
df['Difficulty_Level'] = df['Difficult_Words_%'].apply(Difficult)

In [80]:
def readability(text):
    readability = textstat.flesch_reading_ease(text) #The Flesch Reading Ease formula
    return readability

In [81]:
df['Readability'] = df['Cleaned_Article'].apply(readability)

In [82]:
#convertng into category
def cat(x):
    global ret    
    if (x < 10.0):
        ret = "Professional"  
    elif ((x > 10.0)and (x<= 30.0)):
        ret = "Graduate"
    elif ((x > 30.0)and (x<= 50.0)):
        ret = "College"
    elif ((x > 50.0)and (x<= 60.0)):
        ret = "Grade_10"
    elif ((x > 60.0)and (x<= 70.0)):
        ret = "Grade_8"
    elif ((x > 70.0)and (x<= 80.0)):
        ret = "Grade_7"
    elif ((x > 80.0)and (x<= 90.0)):
        ret = "Grade_6"
    elif (x > 90.0):
        ret = "Grade_5"
    return ret

In [83]:
df['F_Readability'] = df.Readability.apply(cat)

In [84]:
df['Polarity'] = df['Cleaned_Article'].map(lambda text: TextBlob(text).sentiment.polarity)  # range from -1 to 1

In [85]:
df['Sentiment'] = df['Polarity'].apply(lambda x: 'Negative' if x <= 0 else ('Neutral' if x <0.1 else 'Positive'))

In [86]:
df['Subjectivity'] = df['Cleaned_Article'].map(lambda text: TextBlob(text).sentiment.subjectivity) 
# Subjective sentences generally refer to personal opinion, emotion or judgment whereas objective refers to factual information. Subjectivity is also a float which lies in the range of [0,1].
# i.e. 0.75 subjectivity refers that mostly it is a public opinion and not a factual information.

In [87]:
df.head()

,Article_Text,Cleaned_Article,Word_Count,Char_Count,Avg_Word_length,Stopwords_Count,numerics_Count,Upper_Count,Syllable_Count,Sentence_Count,Difficult_Words_Count,length,Article_Size,Difficult_Words_%,Difficulty_Level,Readability,F_Readability,Polarity,Sentiment,Subjectivity
0,Henry Mintzberg about some Half-Truths of Mana...,Henry Mintzberg about some Half-Truths of Mana...,950,5489,4.8195329,373,0,50,1361,32,134,5489,Medium,14.1052632,Elementary,58.5500000,Grade_10,0.1870821,Positive,0.4818873


#MBTI

In [88]:
import re
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import one_hot
from nltk.stem.porter import PorterStemmer
from sklearn import preprocessing
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [89]:
MBTI_model = tf.keras.models.load_model('Prediction.h5')

# Check its architecture
MBTI_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 100)          5000000   
_________________________________________________________________
dropout (Dropout)            (None, 250, 100)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 16)                1616      
Total params: 5,082,016
Trainable params: 5,082,016
Non-trainable params: 0
_________________________________________________________________


In [90]:
def preprocess(input):
    
    ps=PorterStemmer()
    
    L=[]

    review = re.sub('[^a-zA-Z]','',input[0])
    review=review.lower()
    review=review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english') ]
    review=''.join(review)
    L.append(review)
    
    oe=[one_hot(words,voc_size) for words in L]
    
    sent_length=2500
    doc = pad_sequences(oe,padding='pre',maxlen = sent_length)
    
    return doc

In [91]:
voc_size = 50000

In [92]:
#convertng into category
def MBTI_mapping(x):
    
    if (x == 0):
        ret = "ENFJ"  
    elif (x == 1 ):
        ret = "ENFP"
    elif (x == 2 ):
        ret = "ENTJ"
    elif (x == 3 ):
        ret = "ENTP"
    elif (x == 4 ):
        ret = "ESFJ"
    elif (x == 5 ):
        ret = "ESFP"
    elif (x == 6 ):
        ret = "ESTJ"
    elif (x == 7 ):
        ret = "ESTP"
    elif (x == 8 ):
        ret = "INFJ"
    elif (x == 9 ):
        ret = "INFP"
    elif (x == 10 ):
        ret = "INTJ"
    elif (x == 11 ):
        ret = "INTP"
    elif (x == 12 ):
        ret = "ISFJ"
    elif (x == 13 ):
        ret = "ISFP"
    elif (x == 14 ):
        ret = "ISTJ"
    elif (x == 15 ):
        ret = "ISTP"
    
    return ret

In [93]:
def MBTI(input):
    pred=preprocess(input)
    y_pred=MBTI_model.predict(pred)
    y_pred=np.argmax(y_pred)
    y_pred = MBTI_mapping(y_pred)
    return (y_pred)

In [94]:
df['MBTI'] = df['Cleaned_Article'].apply(MBTI)

In [95]:
df.head()

,Article_Text,Cleaned_Article,Word_Count,Char_Count,Avg_Word_length,Stopwords_Count,numerics_Count,Upper_Count,Syllable_Count,Sentence_Count,Difficult_Words_Count,length,Article_Size,Difficult_Words_%,Difficulty_Level,Readability,F_Readability,Polarity,Sentiment,Subjectivity,MBTI
0,Henry Mintzberg about some Half-Truths of Mana...,Henry Mintzberg about some Half-Truths of Mana...,950,5489,4.8195329,373,0,50,1361,32,134,5489,Medium,14.1052632,Elementary,58.5500000,Grade_10,0.1870821,Positive,0.4818873,INTJ


#Domain classification

In [96]:
#convertng into category
def domain_mapping(x):
    if (x == 0):
        ret = "Finance"  
    elif (x == 1 ):
        ret = "Leadership"
    elif (x == 2 ):
        ret = "Marketing"
    elif (x == 3 ):
        ret = "ENStrategyTP"
    elif (x == 4 ):
        ret = "Technology"
    return ret     

Not to run both model 2 and 4. Run only one model

#Model4

In [ ]:
#model_4_path = "/content/drive/MyDrive/AI-ML/models/model_4_path/"
model_4_path = "/content/drive/MyDrive/capstone/model_4_path" #Ujjval

loaded_model_4 = tf.keras.models.load_model(model_4_path)
loaded_model_4.summary()

Model: "model_4_token_and_char_embeddings"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
token_input (InputLayer)        [(None,)]            0                                            
__________________________________________________________________________________________________
char_vectorizer (TextVectorizat (None, 8908)         0           char_input[0][0]                 
__________________________________________________________________________________________________
universal_sentence_encoder (Ker (None, 512)          256797824   token_input[0][0]                
__________________________________________________________________

In [ ]:
# Make function to split sentences into characters
def split_chars(text):
  return " ".join(list(text))

# Split sequence-level data splits into character-level data splits
predict_chars = [split_chars(sentence) for sentence in article_to_predict]

In [ ]:
predict_token_data = tf.data.Dataset.from_tensor_slices(([article_to_predict], ))
predict_char_data = tf.data.Dataset.from_tensor_slices((predict_chars, ))
predict_dataset = tf.data.Dataset.zip((predict_token_data, predict_char_data))

predict_char_token_dataset = tf.data.Dataset.zip((predict_dataset,))
predict_char_token_dataset = predict_char_token_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
loaded_model_4_pred_probs = loaded_model_4.predict(predict_char_token_dataset)
loaded_model_4_pred = tf.argmax(loaded_model_4_pred_probs, axis=1)
loaded_model_4_pred

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>

In [ ]:
predicted_domain_model_4 = domain_mapping(loaded_model_4_pred)
predicted_domain_model_4

'Finance'

In [ ]:
#Append the Domain in a new column
df['Predicted_Domain'] = predicted_domain_model_4
df

,Article_Text,Cleaned_Article,Word_Count,Char_Count,Avg_Word_length,Stopwords_Count,numerics_Count,Upper_Count,Syllable_Count,Sentence_Count,Difficult_Words_Count,length,Article_Size,Difficult_Words_%,Readability,F_Readability,Polarity,Sentiment,Subjectivity,MBTI,Predicted_Domain
0,"July\t\t\t\t\t\t\t\t25, 2017\t\t\t\t\t\t\t\t\t...","The past few weeks, our social feeds have been...",1349,8832,5.5478132,436,6,27,2233,58,264,8832,Large,19.5700519,39.3700000,College,0.1058949,Positive,0.4515363,INTJ,Finance


#Model2

In [97]:
#model_2_path = "/content/drive/MyDrive/AI-ML/models/model_2_path/"
model_2_path = "/content/drive/MyDrive/capstone/model_2_path" #Ujjval

loaded_model_2 = tf.keras.models.load_model(model_2_path)
loaded_model_2.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None,)]                 0         
_________________________________________________________________
universal_sentence_encoder ( (None, 512)               256797824 
_________________________________________________________________
dense_10 (Dense)             (None, 128)               65664     
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 645       
Total params: 256,864,133
Trainable params: 66,309
Non-trainable params: 256,797,824
_________________________________________________________________


In [98]:
loaded_model_2_pred_probs = loaded_model_2.predict(df['Cleaned_Article'])
loaded_model_2_pred = tf.argmax(loaded_model_2_pred_probs, axis=1)
loaded_model_2_pred

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>

In [99]:
predicted_domain_model_2 = domain_mapping(loaded_model_2_pred)
predicted_domain_model_2

'Leadership'

In [100]:
#Append the Domain in a new column
df['Predicted_Domain'] = predicted_domain_model_2
df

,Article_Text,Cleaned_Article,Word_Count,Char_Count,Avg_Word_length,Stopwords_Count,numerics_Count,Upper_Count,Syllable_Count,Sentence_Count,Difficult_Words_Count,length,Article_Size,Difficult_Words_%,Difficulty_Level,Readability,F_Readability,Polarity,Sentiment,Subjectivity,MBTI,Predicted_Domain
0,Henry Mintzberg about some Half-Truths of Mana...,Henry Mintzberg about some Half-Truths of Mana...,950,5489,4.8195329,373,0,50,1361,32,134,5489,Medium,14.1052632,Elementary,58.5500000,Grade_10,0.1870821,Positive,0.4818873,INTJ,Leadership


In [101]:
df.to_csv("July3_Predict_ForNewArticle.csv",index=False)

#Writer Recommendation

In [46]:
# used to supress display of warnings
import warnings

# Pandas is used for data manipulation and analysis
import pandas as pd 

# Numpy is used for large, multi-dimensional arrays and matrices, along with mathematical operators on these arrays
import numpy as np

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# os is used to provide a way of using operating system dependent functionality
# We use it for setting working folder
#import os


In [ ]:
#os.chdir('/content/drive/MyDrive/Colab Notebooks')

In [102]:
import pandas as pd
#df = pd.read_csv('Final_articles_Train_Readability.csv', engine="python")
df = pd.read_csv('28June_Predict_For_All_Articles.csv', engine="python")
df.tail(1)

,Domain,Cleaned_Article,Cleaned_Headline,Cleaned_Author,Word_Count,Article_Size,Char_Count,Avg_Word_length,Stopwords_Count,numerics_Count,Upper_Count,Syllable_Count,Sentence_Count,Difficult_Words_Count,length,Difficult_Words_%,Difficulty_Level,Readability,F_Readability,Polarity,Sentiment,Subjectivity,MBTI
29630,Technology,Mark my word A combination of airplane and mot...,"It's a Bird. It's a Plane. No, It's a Flying C...",Kim Lachance Shandrow,442,Small,2767,5.3348624,128,5,5,690,12,99,2767,22.3981900,Elementary,34.7300000,College,0.0477891,Neutral,0.5062358,INTJ


In [103]:
input = pd.read_csv('July3_Predict_ForNewArticle.csv', engine="python")
input.head(1)

,Article_Text,Cleaned_Article,Word_Count,Char_Count,Avg_Word_length,Stopwords_Count,numerics_Count,Upper_Count,Syllable_Count,Sentence_Count,Difficult_Words_Count,length,Article_Size,Difficult_Words_%,Difficulty_Level,Readability,F_Readability,Polarity,Sentiment,Subjectivity,MBTI,Predicted_Domain
0,Henry Mintzberg about some Half-Truths of Mana...,Henry Mintzberg about some Half-Truths of Mana...,950,5489,4.8195329,373,0,50,1361,32,134,5489,Medium,14.1052632,Elementary,58.5500000,Grade_10,0.1870821,Positive,0.4818873,INTJ,Leadership


In [104]:
input['Domain']=input['Predicted_Domain']
input['Cleaned_Author'] = 'Predicted Author'
input.head()

,Article_Text,Cleaned_Article,Word_Count,Char_Count,Avg_Word_length,Stopwords_Count,numerics_Count,Upper_Count,Syllable_Count,Sentence_Count,Difficult_Words_Count,length,Article_Size,Difficult_Words_%,Difficulty_Level,Readability,F_Readability,Polarity,Sentiment,Subjectivity,MBTI,Predicted_Domain,Domain,Cleaned_Author
0,Henry Mintzberg about some Half-Truths of Mana...,Henry Mintzberg about some Half-Truths of Mana...,950,5489,4.8195329,373,0,50,1361,32,134,5489,Medium,14.1052632,Elementary,58.5500000,Grade_10,0.1870821,Positive,0.4818873,INTJ,Leadership,Leadership,Predicted Author


In [105]:
df1 = pd.concat([df, input], axis=0)
df1.reset_index(inplace=True)
df1.reset_index(inplace=True)
df1.tail(5)

,level_0,index,Domain,Cleaned_Article,Cleaned_Headline,Cleaned_Author,Word_Count,Article_Size,Char_Count,Avg_Word_length,Stopwords_Count,numerics_Count,Upper_Count,Syllable_Count,Sentence_Count,Difficult_Words_Count,length,Difficult_Words_%,Difficulty_Level,Readability,F_Readability,Polarity,Sentiment,Subjectivity,MBTI,Article_Text,Predicted_Domain
29627,29627,29627,Technology,Reports swirled this morning that due to frenz...,Apple Reportedly Delays Launch of Jumbo iPad,Geoff Weiss,310,Small,1910,5.1645161,106,6,1,488,5,74,1910,23.8709677,Elementary,9.9700000,Professional,0.0257885,Neutral,0.4423817,INFP,NaN,NaN
29628,29628,29628,Technology,"When it comes to online dating, we're often ou...",Why on Earth Does eHarmony Offer Real-Life Mat...,Laura Entis,527,Medium,3287,5.2690840,207,2,2,830,9,97,3287,18.4060721,Elementary,13.0100000,Graduate,0.1514704,Positive,0.5020838,ESFP,NaN,NaN
29629,29629,29629,Technology,New York is the first American city to receive...,New Yorkers Can Now Park Their Websites at .nyc,Benjamin Kabin,365,Small,2231,5.1150685,129,2,6,571,5,75,2231,20.5479452,Elementary,-2.4300000,Professional,0.2268376,Positive,0.3822650,INTJ,NaN,NaN
29630,29630,29630,Technology,Mark my word A combination of airplane and mot...,"It's a Bird. It's a Plane. No, It's a Flying C...",Kim Lachance Shandrow,442,Small,2767,5.3348624,128,5,5,690,12,99,2767,22.3981900,Elementary,34.7300000,College,0.0477891,Neutral,0.5062358,INTJ,NaN,NaN
29631,29631,0,Leadership,Henry Mintzberg about some Half-Truths of Mana...,NaN,Predicted Author,950,Medium,5489,4.8195329,373,0,50,1361,32,134,5489,14.1052632,Elementary,58.5500000,Grade_10,0.1870821,Positive,0.4818873,INTJ,Henry Mintzberg about some Half-Truths of Mana...,Leadership


In [106]:
OHE_MBTI = pd.get_dummies(df1.MBTI)
New_df = pd.concat([df1, OHE_MBTI], axis=1)
OHE_F_Readbility = pd.get_dummies(New_df.F_Readability)
New_df = pd.concat([New_df, OHE_F_Readbility], axis=1)
OHE_Domain = pd.get_dummies(New_df.Domain)
New_df = pd.concat([New_df, OHE_Domain], axis=1)
OHE_Difficulty_Level = pd.get_dummies(New_df.Difficulty_Level)
New_df = pd.concat([New_df, OHE_Difficulty_Level], axis=1)
#New_df.reset_index(inplace=True)
#New_df = New_df.fillna(int(0))

In [107]:
New_df.head(1)

,level_0,index,Domain,Cleaned_Article,Cleaned_Headline,Cleaned_Author,Word_Count,Article_Size,Char_Count,Avg_Word_length,Stopwords_Count,numerics_Count,Upper_Count,Syllable_Count,Sentence_Count,Difficult_Words_Count,length,Difficult_Words_%,Difficulty_Level,Readability,F_Readability,Polarity,Sentiment,Subjectivity,MBTI,Article_Text,Predicted_Domain,ESFP,INFJ,INFP,INTJ,INTP,College,Grade_10,Grade_5,Grade_6,Grade_7,Grade_8,Graduate,Professional,Finance,Leadership,Marketing,Strategy,Technology,Advanced,Basic,Elementary,Intermediate
0,0,0,Marketing,One of the most common challenges I see client...,Is Launching a New Brand the Right Move for Yo...,Melissa Packham,1013,Large,6384,5.3559322,382,3,2,1676,32,196,6384,19.3484699,Elementary,31.3500000,College,0.1467929,Positive,0.4563647,INTJ,NaN,NaN,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [108]:
from sklearn.metrics.pairwise import cosine_similarity
X = New_df.drop(["level_0","index","Article_Text","Predicted_Domain","Domain", "Cleaned_Article",	"Cleaned_Headline",	"Word_Count",	"Article_Size",	"Char_Count",	"Avg_Word_length",	"Stopwords_Count",	"numerics_Count",	"Upper_Count",	"Syllable_Count",	"Sentence_Count",	"Difficult_Words_Count",	"length",	"Difficult_Words_%", "Difficulty_Level",	"Readability",	"F_Readability",	"Polarity",	"Sentiment",	"Subjectivity",	"MBTI"],axis=1)

In [109]:
X1 = X.drop_duplicates()
X1.shape  #Removing the record with same features for same authors

(16065, 23)

In [110]:
X1.set_index('Cleaned_Author', inplace = True)

In [111]:
indices = pd.Series(X1.index)
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(X1, X1)
cosine_sim

array([[1.  , 1.  , 0.75, ..., 0.25, 0.5 , 0.5 ],
       [1.  , 1.  , 0.75, ..., 0.25, 0.5 , 0.5 ],
       [0.75, 0.75, 1.  , ..., 0.25, 0.5 , 0.5 ],
       ...,
       [0.25, 0.25, 0.25, ..., 1.  , 0.5 , 0.5 ],
       [0.5 , 0.5 , 0.5 , ..., 0.5 , 1.  , 0.5 ],
       [0.5 , 0.5 , 0.5 , ..., 0.5 , 0.5 , 1.  ]])

In [112]:
# function that takes in writer as input and returns the top 5 writers
def recommendations(Cleaned_Author, cosine_sim = cosine_sim):
    
    recommended_authors = []
    
    # gettin the index of the authors that matches the given author
    idx = indices[indices == Cleaned_Author].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 5 most similar authors
    top_5_indexes = list(score_series.iloc[2:7].index)
    #cosine_score = list (score_series)
    print(top_5_indexes)
    print ()
    
    # populating the list with the titles of the best 5 authors
    for i in top_5_indexes:
        recommended_authors.append(list(X1.index)[i])
        
    return recommended_authors

In [113]:
recommendations('Predicted Author')

[8873, 6076, 7181, 8869, 8867]



['Paul White',
 'Jeffrey Hayzlett',
 'John Ruhlin',
 'Kelsey Ramsden',
 'Kathleen Murphy']

#Test

In [114]:
X1[(X1.shape[0]-1):X1.shape[0]]

,ESFP,INFJ,INFP,INTJ,INTP,College,Grade_10,Grade_5,Grade_6,Grade_7,Grade_8,Graduate,Professional,Finance,Leadership,Marketing,Strategy,Technology,Advanced,Basic,Elementary,Intermediate
Cleaned_Author,,,,,,,,,,,,,,,,,,,,,,
Predicted Author,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0


In [121]:
X1.loc['Paul White']

,ESFP,INFJ,INFP,INTJ,INTP,College,Grade_10,Grade_5,Grade_6,Grade_7,Grade_8,Graduate,Professional,Finance,Leadership,Marketing,Strategy,Technology,Advanced,Basic,Elementary,Intermediate
Cleaned_Author,,,,,,,,,,,,,,,,,,,,,,
Paul White,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
Paul White,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
Paul White,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
Paul White,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
Paul White,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
Paul White,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
